In [77]:
################################  Storage_Optimization.ipynb  ####################################
# Author: Sukhendu Sain
# Description: Main file of codebase. Houses main code
# Data: 23-Nov-2024
#################################################################################

In [96]:
# Import Necessary Libraries, Utils, and Config Files
import utils
from config import *
import pandas as pd
import numpy as np
import os, re
import matplotlib.pyplot as plt
# import importlib
# importlib.reload(utils)

# Data Import and Clean

In [ ]:
#### Read FILE:: (AKINS FoMoCo_Piece_Sales_112222_YTD.xlsx) into Dataframe
df_Akins = utils.read_excel(AKINS_FOMO_FILE_PATH)
df_Akins['Part#'] = df_Akins['Part#'].apply(lambda a: "".join(str(a).split('-')))
if print_df_after_import: utils.print_df(df_Akins, 200) # Print the Dataframe
# ~1-2secs

In [ ]:
#### Read FILE:: (GPARTS Part Measures.xlsx) into Dataframe
df_Gparts = utils.read_excel(GPARTS_FILE_PATH)
if print_df_after_import: utils.print_df(df_Gparts) # Print the Dataframe
# ~50-60secs

In [ ]:
#### Read FILE:: (Wholesale JAN_Oct_Parts_Ranking_Counter_Invoices_All_Brands.xlsx) into Dataframe
df_Wholesale = utils.read_excel(WHOLESALE_FILE_PATH)

# Clean the Wholesale Dataframe
df_Wholesale['Description'] = df_Wholesale['Description'].astype(str)
df_Wholesale = df_Wholesale.drop(columns=[col for col in df_Wholesale.columns if 'Unnamed' in col], inplace=False)
df_Wholesale = df_Wholesale[(df_Wholesale['Vendor'] == 'FOR') | (df_Wholesale['Vendor'] == 'CHR')].reset_index()
df_Wholesale.loc[df_Wholesale['Description'].apply(lambda x: len(x.split("      ")) > 1), 'Avg. Cost'] = df_Wholesale['Description'].apply(lambda x: [i for i in x.strip().split("      ")][-1])
df_Wholesale.loc[df_Wholesale['Description'].apply(lambda x: len(x.split("      ")) > 1), 'Description'] = df_Wholesale['Description'].apply(lambda x: "     ".join([i for i in x.strip().split("      ")][:-1]))

if print_df_after_import: utils.print_df(df_Wholesale) # Print the Dataframe
# ~12-15secs

In [ ]:
#### Read FILE:: (Service JAN_Oct_Parts_Ranking_ROs_All_Brands.xlsx) into Dataframe
df_Service = utils.read_excel(SERVICE_FILE_PATH)

# Clean the Service Dataframe
df_Service = df_Service.drop(columns=[col for col in df_Service.columns if 'Unnamed' in col], inplace=False)
df_Service = df_Service[(df_Service['Vendor'] == 'FOR') | (df_Service['Vendor'] == 'CHR')].reset_index()

if print_df_after_import: utils.print_df(df_Service, 100) # Print the Dataframe
# ~5-6secs

In [83]:
#### Read FILE:: (Counter Pad) into Dataframe

# Data Processing & Calculation

### Make a Big Final Dataframe

In [ ]:
# It will have the Columns - 'Part Number', 'Part Desc.', 'Active', 'Sold (Pcs.)', '0Dimensions', 'Length/Depth', 'Width', 'Height', 'Zone', 'Storage Type', 'Sub Storage', 'Number of Storage needed'
# It will have all the rows with common part nos. from all 4 Files, having Appropriate Sold Pcs. Values, and Dimensions

main_list = []

gParts_PartNos = set(df_Gparts['Svc Part Number'])

common_part_numbers = gParts_PartNos & set(df_Akins['Part#'])
for pn, pddesc, ac, s, ld, w, h in zip(common_part_numbers, df_Gparts["Svc Part Number Description"], df_Gparts['Is Active?'], df_Akins['Sold Pcs '], df_Gparts['Prod Att - Length'], df_Gparts['Prod Att- Width'], df_Gparts['Prod Att - Height']):
    main_list.append([pn, pddesc, "Akins", ac, s, False, ld, w, h, "", "", "", ""])

common_part_numbers = gParts_PartNos & set(df_Wholesale['Part Number'])
for pn, pddesc, ac, s, ld, w, h in zip(common_part_numbers, df_Gparts["Svc Part Number Description"], df_Gparts['Is Active?'], df_Wholesale['Sold'], df_Gparts['Prod Att - Length'], df_Gparts['Prod Att- Width'], df_Gparts['Prod Att - Height']):
    main_list.append([pn, pddesc, "Wholesale", ac, s, False, ld, w, h, "", "", "", ""])


common_part_numbers = gParts_PartNos & set(df_Service['* indicates a superseded part\nPart Number'])
for pn, pddesc, ac, s, ld, w, h in zip(common_part_numbers, df_Gparts["Svc Part Number Description"], df_Gparts['Is Active?'], df_Service['Qty Sold'], df_Gparts['Prod Att - Length'], df_Gparts['Prod Att- Width'], df_Gparts['Prod Att - Height']):
    main_list.append([pn, pddesc, "Service", ac, s, False, ld, w, h, "", "", "", ""])

df_Main = pd.DataFrame(main_list)
df_Main.columns = ['Part#', 'Part Desc.', 'DataSource', 'Active', 'Sold', '0Dimensions', 'Depth', 'Width', 'Height', 'Zone', 'StorageType', 'SubStorage', 'Num. Storage Required']
df_Main = df_Main.sort_values('Sold', ascending=False).reset_index()
df_Main["Zone"] = df_Main["Zone"].astype(str)
df_Main.loc[(df_Main["Depth"] == 0) | (df_Main["Height"] == 0) | (df_Main["Width"] == 0), "0Dimensions"] = True
df_Main = df_Main[df_Main["0Dimensions"] == False].reset_index()
df_Main.drop(['index', 'level_0'], axis=1, inplace=True)

utils.print_df(df_Main)

### Apply Zoning

In [105]:
## Main Function for Apply Zoning
def Apply_Zoning(df_toBeZoned, Zones=['Red Hot', 'Orange', 'Yellow', 'Green', 'Blue'], thresMultiplier=0.2, soldColName='Sold', zoneColName='Zone'):
    # Initialize Variables
    sold_sum = 0 # Keep sum of all Sold until now in the current zone
    threshold = thresMultiplier * df_toBeZoned[soldColName].sum() # Threshold of Sum of Sold of each Zone
    zoneIndex = 0 # Current Zone Index
    zoneStartIndex = 0 # Current Zone Start Index of the Data

    # Main Loop
    for ind in range(df_toBeZoned.shape[0]): # Loop through each Data
        if sold_sum > threshold: # Check if the Sum exceeds the threshold
            df_toBeZoned.loc[zoneStartIndex:ind, zoneColName] = Zones[zoneIndex] # Set all the Rows from Current zoneStartIndex to now the Current Zone
            zoneStartIndex = ind # Set the zoneStartIndex for next zone to the End of current zone
            zoneIndex = zoneIndex + 1 # Increment the Zone Index
            sold_sum = 0 # Reset the Sold Sum when the current zone ends
        else: 
            sold_sum = sold_sum + df_toBeZoned[soldColName].iloc[ind] # If not exceeding add Sold Sum to the Sold of the current row
    df_toBeZoned.loc[df_toBeZoned[zoneColName] == "", zoneColName] = Zones[-1] #  Set all the leftoever empty zone Rows, to the Last Zone

In [106]:
## Run the Apply_Zoning on df_Main
Apply_Zoning(df_Main)

In [ ]:
## Check each Zones, number of Part Numbers and Sum of Sold of each of their Part Numbers
df_Main.groupby('Zone')['Sold'].sum(), df_Main.groupby('Zone')['Sold'].count()

## Start Storage Assignment

### Function for Storage Assignment

In [108]:
def getStorage(zone, pdesc, depth, width, height):
    # Initialize the empty Variables
    storageType = ""
    subStorage = ""

    # For Specialty Items
    

    # For High Density Drawers: If a Zone is Red Hot and meets the maximum dimensions criterion
    if (zone == "Red Hot") & (depth < 24) & (height < 6) & (width < 12):
        storageType = "High Density Drawers" # Set Storage Type accordingly
        if (width < 9): # If the depth is less than 9
            subStorage = "36\" Drawer - 4 Compart"
        if (width < 8):
            subStorage = "48\" Drawer - 6 Compart"
        if (width < 12):
            subStorage = "48\" Drawer - 4 Compart"
    elif (depth < 24) & (height < 15) & (width < 48): # For Clip Shelving:
        storageType = "Clip Shelving" # Set Storage Type accordingly
        if (depth < 12):  # If the depth is less than 12
            subStorage = "12\" Deep - "
        elif (depth < 18):
            subStorage = "18\" Deep - "
        elif (depth < 24):
            subStorage = "24\" Deep - "
        if (width < 36):
            subStorage += "36\" Wide Shelf"
        elif (width < 48):
            subStorage += "48\" Wide Shelf"
    elif (depth < 96) & (height > 12) & (width < 96): # For Bulk Shelving
        storageType = "Bulk Storage" # Set Storage Type accordingly
        if (depth < 24): # If the depth is less than 24
            subStorage = "24\" Deep - "
        elif (depth < 36):
            subStorage = "36\" Deep - "
        elif (depth < 42):
            subStorage = "42\" Deep - "
        elif (depth < 48):
            subStorage = "48\" Deep - "
        elif (depth < 72):
            subStorage = "72\" Deep - "
        elif (depth < 96):
            subStorage = "96\" Deep - "
        if (width < 48):
            subStorage += "48\" Wide Shelf"
        elif (width < 72):
            subStorage += "96\" Wide Shelf"
        elif (width < 96):
            subStorage += "96\" Wide Shelf"

    return storageType, subStorage # Return the Values

#### Apply the Storage Function

In [109]:
for i in range(df_Main.shape[0]):
    # Set the Dimensions of the Data into Variables
    depth = df_Main.loc[i, "Depth"]
    height = df_Main.loc[i, "Height"]
    width = df_Main.loc[i, "Width"]

    zone = df_Main.loc[i, "Zone"]
    pdesc = df_Main.loc[i, "Part Desc."]

    # If any dimension is zero, set empty Storage
    if df_Main.loc[i, "0Dimensions"] == True:
        df_Main.loc[i, "StorageType"] = ""
        df_Main.loc[i, "SubStorage"] = ""
        continue

    # Set Storage of the Parts
    df_Main.loc[i, "StorageType"], df_Main.loc[i, "SubStorage"] = getStorage(zone, pdesc, depth, width, height)


In [ ]:
utils.print_df(df_Main)